In [61]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functions import *
from Scores import *
from create_dataframe import * 
from create_dataframe_regression import * 
import time 

In [62]:
#ON VA APPRENDRE SUR LES 8031 PREMIERES PASSES ET PREVOIR SUR LES 2008 AUTRES

In [63]:
data = pd.read_csv("dataframe_regression.csv")
data  = data.sort_values("passe_id")

In [64]:
col = data.columns.tolist()
col = col[1:]
data = data[col]

In [65]:
Y = data["passe"]
data = data.drop(["passe"], 1)
data = data.drop(["receiver_id"], 1)

In [66]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
 
#scaler = StandardScaler()
method = LogisticRegression()

n_passes = 10039
n_passes_train = int(0.7*n_passes) 
n_passes_test = n_passes - n_passes_train

#data = scaler.fit_transform (data) 
X_train = data [:(n_passes_train*14)]
y_train = Y[:(n_passes_train*14)]
X_test = data [(n_passes_train*14):]
y_test = Y[(n_passes_train*14):]
#X_train, X_test, y_train, y_test = train_test_split(data, Y, test_size=0.2, random_state=37)

In [67]:
method.fit(X_train, y_train)
proba = method.predict_proba (X_test)
pred = method.predict (X_test)
score = method.score(X_test, y_test)
score

/usr/local/insa/anaconda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9280971352684501

In [68]:
np.sum(pred)

280

In [69]:
np.sum(y_test)

3012

In [70]:
#on recupere dans result toutes les probas de prédire une passe 
result = np.zeros(len(proba))
for i in range(len(proba)):
    result[i] = proba[i][1]

In [71]:
#on recupere dans prediction_indice les indices des lignes du dataframe test ou il y a la proba max 
prediction_indice = np.zeros(n_passes_test)
for i in range(n_passes_test):
    prediction_indice[i] = int(np.argmax(result[i*14 + 1 : (i+1) * 14 + 1]) + (i*14 + 1))

In [72]:
prediction_indice

array([1.3000e+01, 2.7000e+01, 3.4000e+01, ..., 4.2136e+04, 4.2142e+04,
       4.2165e+04])

In [73]:
#on recupere dans prediction les receveurs potentiels qui ont le plus de chance de recevoir la passe  
prediction = np.zeros(n_passes_test)
count = 0
for i in prediction_indice:
    prediction[count] = X_test.iloc[int(i)]["receveur_potentiel"] 
    count += 1

In [74]:
prediction

array([11.,  3.,  3., ..., 13.,  3.,  1.])

In [75]:
verif = pd.read_csv("dataframe.csv")
verif = verif [n_passes_train:]
taux_reussite = np.mean ((verif["receiver_id"] - prediction)%14 == 0)
taux_reussite

0.3761620185922975